In [1]:

!pip install -U fortran-magic
import sys; sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

%reload_ext fortranmagic

mpl.rc('figure', figsize=(12, 7))

ran_the_first_cell = True

jan2017 = pd.to_datetime(['2017-01-03 00:00:00+00:00',
 '2017-01-04 00:00:00+00:00',
 '2017-01-05 00:00:00+00:00',
 '2017-01-06 00:00:00+00:00',
 '2017-01-09 00:00:00+00:00',
 '2017-01-10 00:00:00+00:00',
 '2017-01-11 00:00:00+00:00',
 '2017-01-12 00:00:00+00:00',
 '2017-01-13 00:00:00+00:00',
 '2017-01-17 00:00:00+00:00',
 '2017-01-18 00:00:00+00:00',
 '2017-01-19 00:00:00+00:00',
 '2017-01-20 00:00:00+00:00',
 '2017-01-23 00:00:00+00:00',
 '2017-01-24 00:00:00+00:00',
 '2017-01-25 00:00:00+00:00',
 '2017-01-26 00:00:00+00:00',
 '2017-01-27 00:00:00+00:00',
 '2017-01-30 00:00:00+00:00',
 '2017-01-31 00:00:00+00:00',
 '2017-02-01 00:00:00+00:00'])
calendar = jan2017.values.astype('datetime64[D]')

event_dates = pd.to_datetime(['2017-01-06 00:00:00+00:00', 
                             '2017-01-07 00:00:00+00:00', 
                             '2017-01-08 00:00:00+00:00']).values.astype('datetime64[D]')
event_values = np.array([10, 15, 20])

/usr/local/lib/python3.7/dist-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [2]:
from io import BytesIO
import os
from urllib.parse import urlencode

import requests
import numpy as np
import pandas as pd


def read_avocadata(start_date, end_date, cache_loc='avocadata.html'):
    """Download avocado data to a dataframe.
    Parameters
    ----------
    """
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    base_url = 'https://www.marketnews.usda.gov/mnp/fv-report-retail'
    query_params = {
        'class': ['FRUITS'],
        'commodity': ['AVOCADOS'],
        'compareLy': ['No'],
        'endDate': [end_date.strftime("%m/%d/%Y")],
        'format': ['excel'],
        'organic': ['ALL'],
        'portal': ['fv'],
        'region': ['ALL'],
        'repDate': [start_date.strftime("%m/%d/%Y")],
        'type': ['retail'],
    }

    url = base_url + '?' + urlencode(query_params, doseq=1)

    if not os.path.exists(cache_loc):
        resp = requests.get(url, stream=True)
        resp.raise_for_status()

        with open(cache_loc, 'wb') as f:
            for block in resp.iter_content(chunk_size=4096):
                f.write(block)
        f.close()

    with open(cache_loc, 'rb') as f:
        frame = pd.read_html(f, header=0)[0]

    # Cleanup
    frame = frame[frame['Unit'] == 'each']
    frame['Organic'] = (frame['Organic'] == 'Y')
    frame['Variety'].replace(
        {'VARIOUS GREENSKIN VARIETIES': 'GREENSKIN'},
        inplace=True,
    )
    frame['Date'] = pd.to_datetime(frame['Date'].values, utc=True)

    frame['Region'] = frame['Region'].str.replace(' U.S.', '')
    frame['Region'] = frame['Region'].str.replace(' ', '_')

    # Drop useless columns.
    return frame.drop(
        ['Class', 'Commodity', 'Environment', 'Unit', '% Marked Local'],
        axis=1,
    )

In [3]:
from io import BytesIO
import os
from urllib.parse import urlencode

import requests
import numpy as np
import pandas as pd


def read_avocadata(start_date, end_date, cache_loc='avocadata.html'):
    """Download avocado data to a dataframe.
    Parameters
    ----------
    """
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    base_url = 'https://www.marketnews.usda.gov/mnp/fv-report-retail'
    query_params = {
        'class': ['FRUITS'],
        'commodity': ['AVOCADOS'],
        'compareLy': ['No'],
        'endDate': [end_date.strftime("%m/%d/%Y")],
        'format': ['excel'],
        'organic': ['ALL'],
        'portal': ['fv'],
        'region': ['ALL'],
        'repDate': [start_date.strftime("%m/%d/%Y")],
        'type': ['retail'],
    }

    url = base_url + '?' + urlencode(query_params, doseq=1)

    if not os.path.exists(cache_loc):
        resp = requests.get(url, stream=True)
        resp.raise_for_status()

        with open(cache_loc, 'wb') as f:
            for block in resp.iter_content(chunk_size=4096):
                f.write(block)
        f.close()

    with open(cache_loc, 'rb') as f:
        frame = pd.read_html(f, header=0)[0]

    # Cleanup
    frame = frame[frame['Unit'] == 'each']
    frame['Organic'] = (frame['Organic'] == 'Y')
    frame['Variety'].replace(
        {'VARIOUS GREENSKIN VARIETIES': 'GREENSKIN'},
        inplace=True,
    )
    frame['Date'] = pd.to_datetime(frame['Date'].values, utc=True)

    frame['Region'] = frame['Region'].str.replace(' U.S.', '')
    frame['Region'] = frame['Region'].str.replace(' ', '_')

    # Drop useless columns.
    return frame.drop(
        ['Class', 'Commodity', 'Environment', 'Unit', '% Marked Local'],
        axis=1,
    )

In [4]:
## Needed Libraries.
from urllib.parse import urlencode 
import pandas as pd

import requests

## In this case, we will obtain TOMATOES data.
base_url = 'https://www.marketnews.usda.gov/mnp/fv-report-retail'
query_params = {
  'class': ['VEGETABLES'], ## Change Fruit with Vegetables
  'commodity': ['TOMATOES'], ## Change Avocados with Tomatoes
  'compareLy': ['No'],
  'endDate': ['01/01/2021'], ## Hardcoding end Date
  'repDate': ['01/01/2020'], ## Hardcoding rep Date
  'format': ['excel'],
  'organic': ['ALL'],
  'portal': ['fv'],
  'region': ['ALL'],
  'type': ['retail'],
}

## Build URL to send request.
url = base_url + '?' + urlencode(query_params, doseq=1)

## Request data from url
resp = requests.get(url, stream=True)

## Create File with data.
with open('data.html', 'wb') as f:
            for block in resp.iter_content(chunk_size=4096):
                f.write(block)
            f.close()

## Read data from file.
tomatoes_data = pd.read_html('data.html', header=0)[0]

In [5]:
mpl.rc('figure', figsize=(12, 7))

ran_the_first_cell = True

jan2021 = pd.to_datetime(['2021-01-03 00:00:00+00:00',
 '2021-01-04 00:00:00+00:00',
 '2021-01-05 00:00:00+00:00',
 '2021-01-06 00:00:00+00:00',
 '2021-01-09 00:00:00+00:00',
 '2021-01-10 00:00:00+00:00',
 '2021-01-11 00:00:00+00:00',
 '2021-01-12 00:00:00+00:00',
 '2021-01-13 00:00:00+00:00',
 '2021-01-17 00:00:00+00:00',
 '2021-01-18 00:00:00+00:00',
 '2021-01-19 00:00:00+00:00',
 '2021-01-20 00:00:00+00:00',
 '2021-01-23 00:00:00+00:00',
 '2021-01-24 00:00:00+00:00',
 '2021-01-25 00:00:00+00:00',
 '2021-01-26 00:00:00+00:00',
 '2021-01-27 00:00:00+00:00',
 '2021-01-30 00:00:00+00:00',
 '2021-01-31 00:00:00+00:00',
 '2021-02-01 00:00:00+00:00'])
calendarS = jan2021.values.astype('datetime64[D]')

event_datesS = pd.to_datetime(['2021-01-06 00:00:00+00:00', 
                             '2021-01-07 00:00:00+00:00', 
                             '2021-01-08 00:00:00+00:00']).values.astype('datetime64[D]')
event_valuesS = np.array([25, 30, 35])

<center>
  <h1>The PyData Toolbox</h1>
  <h3>Scott Sanderson (Twitter: @scottbsanderson, GitHub: ssanderson)</h3>
  <h3><a href="https://github.com/ssanderson/pydata-toolbox">https://github.com/ssanderson/pydata-toolbox</a></h3>
</center>

# About Me:

<img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/me.jpg?raw=1" alt="Drawing" style="width: 300px;"/>

- Senior Engineer at [Quantopian](www.quantopian.com)
- Background in Mathematics and Philosophy
- **Twitter:** [@scottbsanderson](https://twitter.com/scottbsanderson)
- **GitHub:** [ssanderson](github.com/ssanderson)

## Outline

- Built-in Data Structures
- Numpy `array`
- Pandas `Series`/`DataFrame`
- Plotting and "Real-World" Analyses

# Data Structures

> Rule 5. Data dominates. If you've chosen the right data structures and organized things well, the algorithms
will almost always be self-evident. Data structures, not algorithms, are central to programming.

- *Notes on Programming in C*, by Rob Pike.

# Lists

In [6]:
assert ran_the_first_cell, "Oh noes!"

In [7]:
l = [1, 'two', 3.0, 4, 5.0, "six"]
l

[1, 'two', 3.0, 4, 5.0, 'six']

In [12]:
lS = ['SEVEN', 3.0, 4,7.0, 'SIX', 5]
lS

['SEVEN', 3.0, 4, 7.0, 'SIX', 5]

In [11]:
# Lists can be indexed like C-style arrays.
first = l[0]
second = l[1]
print("first:", first)
print("second:", second)

first: 1
second: two


In [13]:
firstS = lS[0]
secondS = lS[1]
print("first:", firstS)
print("second:", secondS)

first: SEVEN
second: 3.0


In [ ]:
# Negative indexing gives elements relative to the end of the list.
last = l[-1]
penultimate = l[-2]
print("last:", last)
print("second to last:", penultimate)

last: six
second to last: 5.0


In [14]:
lastS = lS[-1]
penultimateS = lS[-2]
print("last:", lastS)
print("second to last:", penultimateS)

last: 5
second to last: SIX


In [16]:
# Lists can also be sliced, which makes a copy of elements between 
# start (inclusive) and stop (exclusive)
sublist = l[1:3]
sublist

['two', 3.0]

In [15]:
sublistS = lS[1:3]
sublistS

[3.0, 4]

In [17]:
# l[:N] is equivalent to l[0:N].
first_three = l[:3]
first_three

[1, 'two', 3.0]

In [18]:
first_threeS = lS[:3]
first_threeS

['SEVEN', 3.0, 4]

In [19]:
# l[3:] is equivalent to l[3:len(l)].
after_three = l[3:]
after_three

[4, 5.0, 'six']

In [20]:
after_threeS = lS[3:]
after_threeS

[7.0, 'SIX', 5]

In [21]:
# There's also a third parameter, "step", which gets every Nth element.
l = ['a', 'b', 'c', 'd', 'e', 'f', 'g','h']
l[1:7:2]

['b', 'd', 'f']

In [23]:
lS = ['I', 'J', 'K', 'L', 'M', 'N', 'Ñ','O']
lS[1:7:2]

['J', 'L', 'N']

In [24]:
# This is a cute way to reverse a list.
l[::-1]

['h', 'g', 'f', 'e', 'd', 'c', 'b', 'a']

In [25]:
lS[::-1]

['O', 'Ñ', 'N', 'M', 'L', 'K', 'J', 'I']

In [26]:
# Lists can be grown efficiently (in O(1) amortized time).
l = [1, 2, 3, 4, 5]
print("Before:", l)
l.append('six')
print("After:", l)

Before: [1, 2, 3, 4, 5]
After: [1, 2, 3, 4, 5, 'six']


In [27]:
lS = [1, 2, 3, 4, 5]
print("Before:", lS)
lS.append('NINE')
print("After:", lS)

Before: [1, 2, 3, 4, 5]
After: [1, 2, 3, 4, 5, 'NINE']


In [28]:
# Comprehensions let us perform elementwise computations.
l = [1, 2, 3, 4, 5]
[x * 2 for x in l]

[2, 4, 6, 8, 10]

In [29]:
lS = [9, 8, 7, 6, 5]
[x * 6 for x in lS]

[54, 48, 42, 36, 30]

## Review: Python Lists

- Zero-indexed sequence of arbitrary Python values.
- Slicing syntax: `l[start:stop:step]` copies elements at regular intervals from `start` to `stop`.
- Efficient (`O(1)`) appends and removes from end.
- Comprehension syntax: `[f(x) for x in l if cond(x)]`.

# Dictionaries

In [30]:
# Dictionaries are key-value mappings.
philosophers = {'David': 'Hume', 'Immanuel': 'Kant', 'Bertrand': 'Russell'}
philosophers

{'Bertrand': 'Russell', 'David': 'Hume', 'Immanuel': 'Kant'}

In [40]:
frutas = {'man': 'zana', 'Pe': 'ra', 'pi': 'ña'}
frutas

{'Pe': 'ra', 'man': 'zana', 'pi': 'ña'}

In [33]:
# Like lists, dictionaries are size-mutable.
philosophers['Ludwig'] = 'Wittgenstein'
philosophers

{'Bertrand': 'Russell',
 'David': 'Hume',
 'Immanuel': 'Kant',
 'Ludwig': 'Wittgenstein'}

In [42]:
frutas['man'] = 'go'
frutas

{'PVU': 'P', 'Pe': 'ra', 'man': 'go', 'pi': 'ña'}

In [35]:
del philosophers['David']
philosophers

{'Bertrand': 'Russell', 'Immanuel': 'Kant', 'Ludwig': 'Wittgenstein'}

In [43]:
del frutas['man']
frutas

{'PVU': 'P', 'Pe': 'ra', 'pi': 'ña'}

In [37]:
# No slicing.
philosophers['Bertrand':'Immanuel']

TypeError: ignored

In [44]:
frutas['A':'S']

TypeError: ignored

## Review: Python Dictionaries

- Unordered key-value mapping from (almost) arbitrary keys to arbitrary values.
- Efficient (`O(1)`) lookup, insertion, and deletion.
- No slicing (would require a notion of order).

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/pacino.gif?raw=1" alt="Drawing" style="width: 100%;"/></center>


In [47]:
# Suppose we have some matrices...
a = [[1, 2, 3],
     [2, 3, 4],
     [5, 6, 7],
     [1, 1, 1]]

b = [[1, 2, 3, 4],
     [2, 3, 4, 5]]

In [46]:
A = [[2, 4, 6],
     [8, 10, 12],
     [14, 16, 18],
     [20, 22, 24]]

B = [[3, 6, 9, 12],
     [15, 18, 21, 24]]

In [48]:
def matmul(A, B):
    """Multiply matrix A by matrix B."""
    rows_out = len(A)
    cols_out = len(B[0])
    out = [[0 for col in range(cols_out)] for row in range(rows_out)]
    
    for i in range(rows_out):
        for j in range(cols_out):
            for k in range(len(B)):
                out[i][j] += A[i][k] * B[k][j]
    return out

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/gross.gif?raw=1" alt="Drawing" style="width: 50%;"/></center>


In [49]:
%%time

matmul(a, b)

CPU times: user 26 µs, sys: 0 ns, total: 26 µs
Wall time: 28.8 µs


[[5, 8, 11, 14], [8, 13, 18, 23], [17, 28, 39, 50], [3, 5, 7, 9]]

In [50]:
matmul(A, B)

[[66, 84, 102, 120],
 [174, 228, 282, 336],
 [282, 372, 462, 552],
 [390, 516, 642, 768]]

In [51]:
import random
def random_matrix(m, n):
    out = []
    for row in range(m):
        out.append([random.random() for _ in range(n)])
    return out

randm = random_matrix(2, 3)
randm

[[0.6600340422408046, 0.5699074022582911, 0.08098387675076313],
 [0.8041740103981813, 0.11258431787640788, 0.1881001017346663]]

In [52]:
randmS = random_matrix(5, 3)
randmS

[[0.3319913976348734, 0.06569501124739063, 0.574441049374938],
 [0.7755857960497928, 0.47160214610443096, 0.9405458805437139],
 [0.9642435794761752, 0.7961924431183028, 0.061426125921054675],
 [0.835592153488455, 0.2705694009754842, 0.21244017657546477],
 [0.17102320537706994, 0.40874684534395966, 0.8469946908242532]]

In [53]:
%%time
randa = random_matrix(600, 100)
randb = random_matrix(100, 600)
x = matmul(randa, randb)

CPU times: user 10.7 s, sys: 14.4 ms, total: 10.7 s
Wall time: 10.7 s


In [54]:
%%time
randa = random_matrix(700, 200)
randb = random_matrix(200, 700)
x = matmul(randa, randb)

CPU times: user 28.7 s, sys: 72.3 ms, total: 28.7 s
Wall time: 28.8 s


In [55]:
# Maybe that's not that bad?  Let's try a simpler case.
def python_dot_product(xs, ys):
    return sum(x * y for x, y in zip(xs, ys))

In [56]:
%%fortran
subroutine fortran_dot_product(xs, ys, result)
    double precision, intent(in) :: xs(:)
    double precision, intent(in) :: ys(:)
    double precision, intent(out) :: result
    
    result = sum(xs * ys)
end

In [57]:
list_data = [float(i) for i in range(100000)]
array_data = np.array(list_data)

In [58]:
list_datas = [float(i) for i in range(30000)]
array_datas = np.array(list_data)

In [59]:
%%time
python_dot_product(list_data, list_data)

CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 14.7 ms


333328333350000.0

In [60]:
%%time
python_dot_product(list_datas, list_datas)

CPU times: user 4.71 ms, sys: 0 ns, total: 4.71 ms
Wall time: 4.74 ms


8999550005000.0

In [61]:
%%time
fortran_dot_product(array_data, array_data)

CPU times: user 186 µs, sys: 5 µs, total: 191 µs
Wall time: 501 µs


333328333350000.0

In [62]:
%%time
fortran_dot_product(array_datas, array_datas)

CPU times: user 201 µs, sys: 0 ns, total: 201 µs
Wall time: 206 µs


333328333350000.0

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/sloth.gif?raw=1" alt="Drawing" style="width: 1080px;"/></center>


## Why is the Python Version so Much Slower?

In [63]:
# Dynamic typing.
def mul_elemwise(xs, ys):
    return [x * y for x, y in zip(xs, ys)]

mul_elemwise([1, 2, 3, 4], [1, 2 + 0j, 3.0, 'four'])
#[type(x) for x in _]

[1, (4+0j), 9.0, 'fourfourfourfour']

In [64]:
# Dynamic typing.
def mul_elemwise(xs, ys):
    return [x * y for x, y in zip(xs, ys)]

mul_elemwise([4, 8, 12, 16], [23, 434 + 0j, 8.0, 'four'])
#[type(x) for x in _]

[92,
 (3472+0j),
 96.0,
 'fourfourfourfourfourfourfourfourfourfourfourfourfourfourfourfour']

In [65]:
# Interpretation overhead.
source_code = 'a + b * c'
bytecode = compile(source_code, '', 'eval')
import dis; dis.dis(bytecode)

  1           0 LOAD_NAME                0 (a)
              2 LOAD_NAME                1 (b)
              4 LOAD_NAME                2 (c)
              6 BINARY_MULTIPLY
              8 BINARY_ADD
             10 RETURN_VALUE


In [66]:
source_code = 'a + f * b + d'
bytecode = compile(source_code, '', 'eval')
import dis; dis.dis(bytecode)

  1           0 LOAD_NAME                0 (a)
              2 LOAD_NAME                1 (f)
              4 LOAD_NAME                2 (b)
              6 BINARY_MULTIPLY
              8 BINARY_ADD
             10 LOAD_NAME                3 (d)
             12 BINARY_ADD
             14 RETURN_VALUE


## Why is the Python Version so Slow?
- Dynamic typing means that every single operation requires dispatching on the input type.
- Having an interpreter means that every instruction is fetched and dispatched at runtime.
- Other overheads:
  - Arbitrary-size integers.
  - Reference-counted garbage collection.

> This is the paradox that we have to work with when we're doing scientific or numerically-intensive Python. What makes Python fast for development -- this high-level, interpreted, and dynamically-typed aspect of the language -- is exactly what makes it slow for code execution.

- Jake VanderPlas, [*Losing Your Loops: Fast Numerical Computing with NumPy*](https://www.youtube.com/watch?v=EEUXKG97YRw)

# What Do We Do?

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/runaway.gif?raw=1" alt="Drawing" style="width: 50%;"/></center>

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/thisisfine.gif?raw=1" alt="Drawing" style="width: 1080px;"/></center>

- Python is slow for numerical computation because it performs dynamic dispatch on every operation we perform...

- ...but often, we just want to do the same thing over and over in a loop!

- If we don't need Python's dynamicism, we don't want to pay (much) for it.

- **Idea:** Dispatch **once per operation** instead of **once per element**.

In [67]:
import numpy as np

data = np.array([1, 2, 3, 4])
data

array([1, 2, 3, 4])

In [70]:
dataS = np.array([8, 6, 2, 2])
dataS

array([8, 6, 2, 2])

In [69]:
data + data

array([2, 4, 6, 8])

In [71]:
dataS + dataS

array([16, 12,  4,  4])

In [72]:
%%time
# Naive dot product
(array_data * array_data).sum()

CPU times: user 1.15 ms, sys: 4 µs, total: 1.15 ms
Wall time: 3.44 ms


333328333350000.0

In [73]:
%%time
(array_datas * array_datas).sum()

CPU times: user 1.22 ms, sys: 34 µs, total: 1.25 ms
Wall time: 754 µs


333328333350000.0

In [74]:
%%time
# Built-in dot product.
array_data.dot(array_data)

CPU times: user 1.23 ms, sys: 6.06 ms, total: 7.29 ms
Wall time: 7.36 ms


333328333350000.0

In [75]:
array_data.dot(array_datas)

333328333350000.0

In [76]:
%%time
fortran_dot_product(array_data, array_data)

CPU times: user 181 µs, sys: 0 ns, total: 181 µs
Wall time: 185 µs


333328333350000.0

In [77]:
fortran_dot_product(array_datas, array_datas)

333328333350000.0

In [78]:
# Numpy won't allow us to write a string into an int array.
data[0] = "foo"

ValueError: ignored

In [79]:
data[0] = "jejej"

ValueError: ignored

In [80]:
# We also can't grow an array once it's created.
data.append(3)

AttributeError: ignored

In [81]:
# We **can** reshape an array though.
two_by_two = data.reshape(2, 2)
two_by_two

array([[1, 2],
       [3, 4]])

In [82]:
two_by_two = dataS.reshape(2, 2)
two_by_two

array([[8, 6],
       [2, 2]])

Numpy arrays are:

- Fixed-type

- Size-immutable

- Multi-dimensional

- Fast\*

\* If you use them correctly.

# What's in an Array?

In [83]:
arr = np.array([1, 2, 3, 4, 5, 6], dtype='int16').reshape(2, 3)
print("Array:\n", arr, sep='')
print("===========")
print("DType:", arr.dtype)
print("Shape:", arr.shape)
print("Strides:", arr.strides)
print("Data:", arr.data.tobytes())

Array:
[[1 2 3]
 [4 5 6]]
DType: int16
Shape: (2, 3)
Strides: (6, 2)
Data: b'\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00'


In [84]:
arr = np.array([43, 234, 3423, 23, 231, 2187], dtype='int16').reshape(2, 3)
print("Array:\n", arr, sep='')
print("===========")
print("DType:", arr.dtype)
print("Shape:", arr.shape)
print("Strides:", arr.strides)
print("Data:", arr.data.tobytes())

Array:
[[  43  234 3423]
 [  23  231 2187]]
DType: int16
Shape: (2, 3)
Strides: (6, 2)
Data: b'+\x00\xea\x00_\r\x17\x00\xe7\x00\x8b\x08'


# Core Operations

- Vectorized **ufuncs** for elementwise operations.
- Fancy indexing and masking for selection and filtering.
- Aggregations across axes.
- Broadcasting

# UFuncs

UFuncs (universal functions) are functions that operate elementwise on one or more arrays.

In [85]:
Hdata = np.arange(15).reshape(3, 5)
data

array([1, 2, 3, 4])

In [88]:
dataS = np.arange(32).reshape(4, 8)
dataS

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31]])

In [89]:
# Binary operators.
data * data

array([ 1,  4,  9, 16])

In [90]:
dataS * dataS

array([[  0,   1,   4,   9,  16,  25,  36,  49],
       [ 64,  81, 100, 121, 144, 169, 196, 225],
       [256, 289, 324, 361, 400, 441, 484, 529],
       [576, 625, 676, 729, 784, 841, 900, 961]])

In [91]:
# Unary functions.
np.sqrt(data)

array([1.        , 1.41421356, 1.73205081, 2.        ])

In [92]:
np.sqrt(dataS)

array([[0.        , 1.        , 1.41421356, 1.73205081, 2.        ,
        2.23606798, 2.44948974, 2.64575131],
       [2.82842712, 3.        , 3.16227766, 3.31662479, 3.46410162,
        3.60555128, 3.74165739, 3.87298335],
       [4.        , 4.12310563, 4.24264069, 4.35889894, 4.47213595,
        4.58257569, 4.69041576, 4.79583152],
       [4.89897949, 5.        , 5.09901951, 5.19615242, 5.29150262,
        5.38516481, 5.47722558, 5.56776436]])

In [93]:
# Comparison operations
(data % 3) == 0

array([False, False,  True, False])

In [94]:
(dataS % 3) == 0

array([[ True, False, False,  True, False, False,  True, False],
       [False,  True, False, False,  True, False, False,  True],
       [False, False,  True, False, False,  True, False, False],
       [ True, False, False,  True, False, False,  True, False]])

In [95]:
# Boolean combinators.
((data % 2) == 0) & ((data % 3) == 0)

array([False, False, False, False])

In [96]:
((dataS % 2) == 0) & ((dataS % 3) == 0)

array([[ True, False, False, False, False, False,  True, False],
       [False, False, False, False,  True, False, False, False],
       [False, False,  True, False, False, False, False, False],
       [ True, False, False, False, False, False,  True, False]])

In [97]:
# as of python 3.5, @ is matrix-multiply
data @ data.T

30

In [98]:
dataS @ dataS.T

array([[ 140,  364,  588,  812],
       [ 364, 1100, 1836, 2572],
       [ 588, 1836, 3084, 4332],
       [ 812, 2572, 4332, 6092]])

# UFuncs Review

- UFuncs provide efficient elementwise operations applied across one or more arrays.
- Arithmetic Operators (`+`, `*`, `/`)
- Comparisons (`==`, `>`, `!=`)
- Boolean Operators (`&`, `|`, `^`)
- Trigonometric Functions (`sin`, `cos`)
- Transcendental Functions (`exp`, `log`)

# Selections

We often want to perform an operation on just a subset of our data.

In [99]:
sines = np.sin(np.linspace(0, 3.14, 10))
cosines = np.cos(np.linspace(0, 3.14, 10))
sines

array([0.        , 0.34185385, 0.64251645, 0.86575984, 0.98468459,
       0.98496101, 0.8665558 , 0.64373604, 0.34335012, 0.00159265])

In [101]:
sinesS = np.sin(np.linspace(0, 5.32, 341))
cosinesS = np.cos(np.linspace(3, 3.14, 10))
sinesS

array([ 0.        ,  0.01564642,  0.03128901,  0.04692394,  0.06254738,
        0.07815551,  0.0937445 ,  0.10931054,  0.12484983,  0.14035854,
        0.15583289,  0.17126909,  0.18666336,  0.20201192,  0.21731103,
        0.23255694,  0.24774591,  0.26287423,  0.27793819,  0.2929341 ,
        0.30785829,  0.32270712,  0.33747693,  0.35216413,  0.3667651 ,
        0.38127628,  0.39569411,  0.41001507,  0.42423565,  0.43835236,
        0.45236176,  0.4662604 ,  0.48004489,  0.49371185,  0.50725794,
        0.52067984,  0.53397427,  0.54713796,  0.5601677 ,  0.5730603 ,
        0.5858126 ,  0.59842148,  0.61088384,  0.62319665,  0.63535689,
        0.64736157,  0.65920776,  0.67089256,  0.68241311,  0.69376658,
        0.70495021,  0.71596124,  0.72679699,  0.7374548 ,  0.74793207,
        0.75822622,  0.76833474,  0.77825515,  0.78798502,  0.79752197,
        0.80686367,  0.81600783,  0.82495221,  0.83369462,  0.84223293,
        0.85056503,  0.85868889,  0.86660252,  0.87430399,  0.88

In [102]:
# Slicing works with the same semantics as Python lists.
sines[0]

0.0

In [103]:
sinesS[0]

0.0

In [104]:
sines[:3]  # First three elements  

array([0.        , 0.34185385, 0.64251645])

In [105]:
sinesS[:3]  

array([0.        , 0.01564642, 0.03128901])

In [106]:
sines[5:]  # Elements from 5 on.

array([0.98496101, 0.8665558 , 0.64373604, 0.34335012, 0.00159265])

In [107]:
sinesS[5:]

array([ 0.07815551,  0.0937445 ,  0.10931054,  0.12484983,  0.14035854,
        0.15583289,  0.17126909,  0.18666336,  0.20201192,  0.21731103,
        0.23255694,  0.24774591,  0.26287423,  0.27793819,  0.2929341 ,
        0.30785829,  0.32270712,  0.33747693,  0.35216413,  0.3667651 ,
        0.38127628,  0.39569411,  0.41001507,  0.42423565,  0.43835236,
        0.45236176,  0.4662604 ,  0.48004489,  0.49371185,  0.50725794,
        0.52067984,  0.53397427,  0.54713796,  0.5601677 ,  0.5730603 ,
        0.5858126 ,  0.59842148,  0.61088384,  0.62319665,  0.63535689,
        0.64736157,  0.65920776,  0.67089256,  0.68241311,  0.69376658,
        0.70495021,  0.71596124,  0.72679699,  0.7374548 ,  0.74793207,
        0.75822622,  0.76833474,  0.77825515,  0.78798502,  0.79752197,
        0.80686367,  0.81600783,  0.82495221,  0.83369462,  0.84223293,
        0.85056503,  0.85868889,  0.86660252,  0.87430399,  0.8817914 ,
        0.88906293,  0.89611679,  0.90295127,  0.90956467,  0.91

In [108]:
sines[::2]  # Every other element.

array([0.        , 0.64251645, 0.98468459, 0.8665558 , 0.34335012])

In [109]:
sinesS[::2]

array([ 0.        ,  0.03128901,  0.06254738,  0.0937445 ,  0.12484983,
        0.15583289,  0.18666336,  0.21731103,  0.24774591,  0.27793819,
        0.30785829,  0.33747693,  0.3667651 ,  0.39569411,  0.42423565,
        0.45236176,  0.48004489,  0.50725794,  0.53397427,  0.5601677 ,
        0.5858126 ,  0.61088384,  0.63535689,  0.65920776,  0.68241311,
        0.70495021,  0.72679699,  0.74793207,  0.76833474,  0.78798502,
        0.80686367,  0.82495221,  0.84223293,  0.85868889,  0.87430399,
        0.88906293,  0.90295127,  0.91595539,  0.92806258,  0.93926097,
        0.94953959,  0.95888839,  0.9672982 ,  0.97476079,  0.98126886,
        0.98681603,  0.99139686,  0.99500688,  0.99764255,  0.99930128,
        0.99998145,  0.9996824 ,  0.99840442,  0.99614876,  0.99291763,
        0.9887142 ,  0.98354257,  0.97740782,  0.97031595,  0.9622739 ,
        0.95328956,  0.94337171,  0.93253008,  0.92077527,  0.9081188 ,
        0.89457306,  0.88015132,  0.8648677 ,  0.84873717,  0.83

In [110]:
# More interesting: we can index with boolean arrays to filter by a predicate.
print("sines:\n", sines)
print("sines > 0.5:\n", sines > 0.5)
print("sines[sines > 0.5]:\n", sines[sines > 0.5])

sines:
 [0.         0.34185385 0.64251645 0.86575984 0.98468459 0.98496101
 0.8665558  0.64373604 0.34335012 0.00159265]
sines > 0.5:
 [False False  True  True  True  True  True  True False False]
sines[sines > 0.5]:
 [0.64251645 0.86575984 0.98468459 0.98496101 0.8665558  0.64373604]


In [111]:
print("sinesS:\n", sinesS)
print("sinesS > 0.5:\n", sinesS > 0.5)
print("sinesS[sinesS > 0.5]:\n", sinesS[sinesS > 0.5])

sinesS:
 [ 0.          0.01564642  0.03128901  0.04692394  0.06254738  0.07815551
  0.0937445   0.10931054  0.12484983  0.14035854  0.15583289  0.17126909
  0.18666336  0.20201192  0.21731103  0.23255694  0.24774591  0.26287423
  0.27793819  0.2929341   0.30785829  0.32270712  0.33747693  0.35216413
  0.3667651   0.38127628  0.39569411  0.41001507  0.42423565  0.43835236
  0.45236176  0.4662604   0.48004489  0.49371185  0.50725794  0.52067984
  0.53397427  0.54713796  0.5601677   0.5730603   0.5858126   0.59842148
  0.61088384  0.62319665  0.63535689  0.64736157  0.65920776  0.67089256
  0.68241311  0.69376658  0.70495021  0.71596124  0.72679699  0.7374548
  0.74793207  0.75822622  0.76833474  0.77825515  0.78798502  0.79752197
  0.80686367  0.81600783  0.82495221  0.83369462  0.84223293  0.85056503
  0.85868889  0.86660252  0.87430399  0.8817914   0.88906293  0.89611679
  0.90295127  0.90956467  0.91595539  0.92212187  0.92806258  0.93377608
  0.93926097  0.9445159   0.94953959  0.954

In [112]:
# We index with lists/arrays of integers to select values at those indices.
print(sines)
sines[[0, 4, 7]]

[0.         0.34185385 0.64251645 0.86575984 0.98468459 0.98496101
 0.8665558  0.64373604 0.34335012 0.00159265]


array([0.        , 0.98468459, 0.64373604])

In [113]:
print(sinesS)
sinesS[[0, 4, 7]]

[ 0.          0.01564642  0.03128901  0.04692394  0.06254738  0.07815551
  0.0937445   0.10931054  0.12484983  0.14035854  0.15583289  0.17126909
  0.18666336  0.20201192  0.21731103  0.23255694  0.24774591  0.26287423
  0.27793819  0.2929341   0.30785829  0.32270712  0.33747693  0.35216413
  0.3667651   0.38127628  0.39569411  0.41001507  0.42423565  0.43835236
  0.45236176  0.4662604   0.48004489  0.49371185  0.50725794  0.52067984
  0.53397427  0.54713796  0.5601677   0.5730603   0.5858126   0.59842148
  0.61088384  0.62319665  0.63535689  0.64736157  0.65920776  0.67089256
  0.68241311  0.69376658  0.70495021  0.71596124  0.72679699  0.7374548
  0.74793207  0.75822622  0.76833474  0.77825515  0.78798502  0.79752197
  0.80686367  0.81600783  0.82495221  0.83369462  0.84223293  0.85056503
  0.85868889  0.86660252  0.87430399  0.8817914   0.88906293  0.89611679
  0.90295127  0.90956467  0.91595539  0.92212187  0.92806258  0.93377608
  0.93926097  0.9445159   0.94953959  0.95433081  0.

array([0.        , 0.06254738, 0.10931054])

In [114]:
# Index arrays are often used for sorting one or more arrays.
unsorted_data = np.array([1, 3, 2, 12, -1, 5, 2])

In [116]:
unsorted_dataS = np.array([23, 3, 54, 2, -2, 3, 5327])

In [117]:
sort_indices = np.argsort(unsorted_data)
sort_indices

array([4, 0, 2, 6, 1, 5, 3])

In [118]:
sort_indicesS = np.argsort(unsorted_dataS)
sort_indicesS

array([4, 3, 1, 5, 0, 2, 6])

In [119]:
unsorted_data[sort_indices]

array([-1,  1,  2,  2,  3,  5, 12])

In [120]:
unsorted_dataS[sort_indicesS]

array([  -2,    2,    3,    3,   23,   54, 5327])

In [121]:
market_caps = np.array([12, 6, 10, 5, 6])  # Presumably in dollars?
assets = np.array(['A', 'B', 'C', 'D', 'E'])

In [122]:
market_capsS = np.array([12, 6, 10, 5, 6])  # Presumably in dollars?
assetsS = np.array(['F', 'G', 'H', 'I', 'J'])

In [124]:
# Sort assets by market cap by using the permutation that would sort market caps on ``assets``.
sort_by_mcap = np.argsort(market_caps)
assets[sort_by_mcap]

array(['D', 'B', 'E', 'C', 'A'], dtype='<U1')

In [125]:
sort_by_mcapS = np.argsort(market_capsS)
assetsS[sort_by_mcapS]

array(['I', 'G', 'J', 'H', 'F'], dtype='<U1')

In [126]:
# Indexers are also useful for aligning data.
print("Dates:\n", repr(event_dates))
print("Values:\n", repr(event_values))
print("Calendar:\n", repr(calendar))

Dates:
 array(['2017-01-06', '2017-01-07', '2017-01-08'], dtype='datetime64[D]')
Values:
 array([10, 15, 20])
Calendar:
 array(['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06',
       '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
       '2017-01-13', '2017-01-17', '2017-01-18', '2017-01-19',
       '2017-01-20', '2017-01-23', '2017-01-24', '2017-01-25',
       '2017-01-26', '2017-01-27', '2017-01-30', '2017-01-31',
       '2017-02-01'], dtype='datetime64[D]')


In [127]:
print("Dates:\n", repr(event_datesS))
print("Values:\n", repr(event_valuesS))
print("Calendar:\n", repr(calendarS))

Dates:
 array(['2021-01-06', '2021-01-07', '2021-01-08'], dtype='datetime64[D]')
Values:
 array([25, 30, 35])
Calendar:
 array(['2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06',
       '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12',
       '2021-01-13', '2021-01-17', '2021-01-18', '2021-01-19',
       '2021-01-20', '2021-01-23', '2021-01-24', '2021-01-25',
       '2021-01-26', '2021-01-27', '2021-01-30', '2021-01-31',
       '2021-02-01'], dtype='datetime64[D]')


In [128]:
print("Raw Dates:", event_dates)
print("Indices:", calendar.searchsorted(event_dates))
print("Forward-Filled Dates:", calendar[calendar.searchsorted(event_dates)])

Raw Dates: ['2017-01-06' '2017-01-07' '2017-01-08']
Indices: [3 4 4]
Forward-Filled Dates: ['2017-01-06' '2017-01-09' '2017-01-09']


In [129]:
print("Raw Dates:", event_datesS)
print("Indices:", calendar.searchsorted(event_datesS))
print("Forward-Filled Dates:", calendar[calendarS.searchsorted(event_datesS)])

Raw Dates: ['2021-01-06' '2021-01-07' '2021-01-08']
Indices: [21 21 21]
Forward-Filled Dates: ['2017-01-06' '2017-01-09' '2017-01-09']


On multi-dimensional arrays, we can slice along each axis independently.

In [130]:
data = np.arange(25).reshape(5, 5)
data

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [131]:
dataS = np.arange(64).reshape(8, 8)
dataS

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])

In [132]:
data[:2, :2]  # First two rows and first two columns.

array([[0, 1],
       [5, 6]])

In [134]:
dataS[:3, :6]

array([[ 0,  1,  2,  3,  4,  5],
       [ 8,  9, 10, 11, 12, 13],
       [16, 17, 18, 19, 20, 21]])

In [135]:
data[:2, [0, -1]]  # First two rows, first and last columns.

array([[0, 4],
       [5, 9]])

In [136]:
dataS[:2, [0, -1]]

array([[ 0,  7],
       [ 8, 15]])

In [137]:
data[(data[:, 0] % 2) == 0]  # Rows where the first column is divisible by two.

array([[ 0,  1,  2,  3,  4],
       [10, 11, 12, 13, 14],
       [20, 21, 22, 23, 24]])

In [138]:
dataS[(dataS[:, 0] % 2) == 0]

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])

# Selections Review

- Indexing with an integer removes a dimension.
- Slicing operations work on Numpy arrays the same way they do on lists.
- Indexing with a boolean array filters to True locations.
- Indexing with an integer array selects indices along an axis.
- Multidimensional arrays can apply selections independently along different axes.

## Reductions

Functions that reduce an array to a scalar.

$Var(X) = \frac{1}{N}\sqrt{\sum_{i=1}^N (x_i - \bar{x})^2}$

In [139]:
def variance(x):
    return ((x - x.mean()) ** 2).sum() / len(x)

In [ ]:
variance(np.random.standard_normal(1000))

1.0102556393337563

In [141]:
variance(np.random.standard_normal(2000))

1.0074366963918004

- `sum()` and `mean()` are both **reductions**.

- In the simplest case, we use these to reduce an entire array into a single value...

In [142]:
data = np.arange(30)
data.mean()

14.5

In [143]:
dataS = np.arange(40)
dataS.mean()

19.5

- ...but we can do more interesting things with multi-dimensional arrays.

In [144]:
data = np.arange(30).reshape(3, 10)
data

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [145]:
dataS = np.arange(144).reshape(12, 12)
dataS

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11],
       [ 12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23],
       [ 24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35],
       [ 36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47],
       [ 48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71],
       [ 72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83],
       [ 84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95],
       [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107],
       [108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119],
       [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131],
       [132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143]])

In [146]:
data.mean()

14.5

In [147]:
dataS.mean()

71.5

In [148]:
data.mean(axis=0)

array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19.])

In [149]:
dataS.mean(axis=0)

array([66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.])

In [150]:
data.mean(axis=1)

array([ 4.5, 14.5, 24.5])

In [151]:
dataS.mean(axis=1)

array([  5.5,  17.5,  29.5,  41.5,  53.5,  65.5,  77.5,  89.5, 101.5,
       113.5, 125.5, 137.5])

## Reductions Review

- Reductions allow us to perform efficient aggregations over arrays.
- We can do aggregations over a single axis to collapse a single dimension.
- Many built-in reductions (`mean`, `sum`, `min`, `max`, `median`, ...).

# Broadcasting

In [152]:
row = np.array([1, 2, 3, 4])
column = np.array([[1], [2], [3]])
print("Row:\n", row, sep='')
print("Column:\n", column, sep='')

Row:
[1 2 3 4]
Column:
[[1]
 [2]
 [3]]


In [155]:
rowS = np.array([6, 7, 8, 9, 10])
columnS = np.array([[1], [2], [3], [4]])
print("Row:\n", rowS, sep='')
print("Column:\n", columnS, sep='')

Row:
[ 6  7  8  9 10]
Column:
[[1]
 [2]
 [3]
 [4]]


In [154]:
row + column

array([[2, 3, 4, 5],
       [3, 4, 5, 6],
       [4, 5, 6, 7]])

In [156]:
rowS + columnS

array([[ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 12],
       [ 9, 10, 11, 12, 13],
       [10, 11, 12, 13, 14]])

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/broadcasting.png?raw=1" alt="Drawing" style="width: 60%;"/></center>

<h5>Source: http://www.scipy-lectures.org/_images/numpy_broadcasting.png</h5>

In [157]:
# Broadcasting is particularly useful in conjunction with reductions.
print("Data:\n", data, sep='')
print("Mean:\n", data.mean(axis=0), sep='')
print("Data - Mean:\n", data - data.mean(axis=0), sep='')

Data:
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]]
Mean:
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
Data - Mean:
[[-10. -10. -10. -10. -10. -10. -10. -10. -10. -10.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 10.  10.  10.  10.  10.  10.  10.  10.  10.  10.]]


In [158]:
print("Data:\n", dataS, sep='')
print("Mean:\n", dataS.mean(axis=0), sep='')
print("Data - Mean:\n", dataS - dataS.mean(axis=0), sep='')

Data:
[[  0   1   2   3   4   5   6   7   8   9  10  11]
 [ 12  13  14  15  16  17  18  19  20  21  22  23]
 [ 24  25  26  27  28  29  30  31  32  33  34  35]
 [ 36  37  38  39  40  41  42  43  44  45  46  47]
 [ 48  49  50  51  52  53  54  55  56  57  58  59]
 [ 60  61  62  63  64  65  66  67  68  69  70  71]
 [ 72  73  74  75  76  77  78  79  80  81  82  83]
 [ 84  85  86  87  88  89  90  91  92  93  94  95]
 [ 96  97  98  99 100 101 102 103 104 105 106 107]
 [108 109 110 111 112 113 114 115 116 117 118 119]
 [120 121 122 123 124 125 126 127 128 129 130 131]
 [132 133 134 135 136 137 138 139 140 141 142 143]]
Mean:
[66. 67. 68. 69. 70. 71. 72. 73. 74. 75. 76. 77.]
Data - Mean:
[[-66. -66. -66. -66. -66. -66. -66. -66. -66. -66. -66. -66.]
 [-54. -54. -54. -54. -54. -54. -54. -54. -54. -54. -54. -54.]
 [-42. -42. -42. -42. -42. -42. -42. -42. -42. -42. -42. -42.]
 [-30. -30. -30. -30. -30. -30. -30. -30. -30. -30. -30. -30.]
 [-18. -18. -18. -18. -18. -18. -18. -18. -18. -18. -18. -18

# Broadcasting Review

- Numpy operations can work on arrays of different dimensions as long as the arrays' shapes are still "compatible".
- Broadcasting works by "tiling" the smaller array along the missing dimension.
- The result of a broadcasted operation is always at least as large in each dimension as the largest array in that dimension.

# Numpy Review

- Numerical algorithms are slow in pure Python because the overhead dynamic dispatch dominates our runtime.

- Numpy solves this problem by:
  1. Imposing additional restrictions on the contents of arrays.
  2. Moving the inner loops of our algorithms into compiled C code.

- Using Numpy effectively often requires reworking an algorithms to use vectorized operations instead of for-loops, but the resulting operations are usually simpler, clearer, and faster than the pure Python equivalent.

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/unicorn.jpg?raw=1" alt="Drawing" style="width: 75%;"/></center>

Numpy is great for many things, but...

- Sometimes our data is equipped with a natural set of **labels**:
  - Dates/Times
  - Stock Tickers
  - Field Names (e.g. Open/High/Low/Close)

- Sometimes we have **more than one type of data** that we want to keep grouped together.
  - Tables with a mix of real-valued and categorical data.

- Sometimes we have **missing** data, which we need to ignore, fill, or otherwise work around.

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/panda-wrangling.gif?raw=1" alt="Drawing" style="width: 75%;"/></center>

<center><img src="https://github.com/ssanderson/pydata-toolbox/blob/master/notebooks/images/pandas_logo.png?raw=1" alt="Drawing" style="width: 75%;"/></center>


Pandas extends Numpy with more complex data structures:

- `Series`: 1-dimensional, homogenously-typed, labelled array.
- `DataFrame`: 2-dimensional, semi-homogenous, labelled table.

Pandas also provides many utilities for: 
- Input/Output
- Data Cleaning
- Rolling Algorithms
- Plotting

# Selection in Pandas

In [159]:
s = pd.Series(index=['a', 'b', 'c', 'd', 'e'], data=[1, 2, 3, 4, 5])
s

a    1
b    2
c    3
d    4
e    5
dtype: int64

In [160]:
sS = pd.Series(index=['f', 'g', 'h', 'i', 'j'], data=[6, 7, 8, 9, 10])
sS

f     6
g     7
h     8
i     9
j    10
dtype: int64

In [161]:
# There are two pieces to a Series: the index and the values.
print("The index is:", s.index)
print("The values are:", s.values)

The index is: Index(['a', 'b', 'c', 'd', 'e'], dtype='object')
The values are: [1 2 3 4 5]


In [162]:
print("The index is:", sS.index)
print("The values are:", sS.values)

The index is: Index(['f', 'g', 'h', 'i', 'j'], dtype='object')
The values are: [ 6  7  8  9 10]


In [163]:
# We can look up values out of a Series by position...
s.iloc[0]

1

In [164]:
sS.iloc[0]

6

In [165]:
# ... or by label.
s.loc['a']

1

In [167]:
sS.loc['g']

7

In [168]:
# Slicing works as expected...
s.iloc[:2]

a    1
b    2
dtype: int64

In [169]:
sS.iloc[:3]

f    6
g    7
h    8
dtype: int64

In [170]:
# ...but it works with labels too!
s.loc[:'c']

a    1
b    2
c    3
dtype: int64

In [172]:
sS.loc[:'h']

f    6
g    7
h    8
dtype: int64

In [173]:
# Fancy indexing works the same as in numpy.
s.iloc[[0, -1]]

a    1
e    5
dtype: int64

In [174]:
sS.iloc[[0, -1]]

f     6
j    10
dtype: int64

In [175]:
# As does boolean masking.
s.loc[s > 2]

c    3
d    4
e    5
dtype: int64

In [176]:
sS.loc[sS > 7]

h     8
i     9
j    10
dtype: int64

In [177]:
# Element-wise operations are aligned by index.
other_s = pd.Series({'a': 10.0, 'c': 20.0, 'd': 30.0, 'z': 40.0})
other_s

a    10.0
c    20.0
d    30.0
z    40.0
dtype: float64

In [178]:
other_sS = pd.Series({'a': 40.0, 's': 50.0, 'd': 60.0, 'f': 70.0})
other_sS

a    40.0
s    50.0
d    60.0
f    70.0
dtype: float64

In [179]:
s + other_s

a    11.0
b     NaN
c    23.0
d    34.0
e     NaN
z     NaN
dtype: float64

In [180]:
sS + other_sS

a     NaN
d     NaN
f    76.0
g     NaN
h     NaN
i     NaN
j     NaN
s     NaN
dtype: float64

In [181]:
# We can fill in missing values with fillna().
(s + other_s).fillna(0.0)

a    11.0
b     0.0
c    23.0
d    34.0
e     0.0
z     0.0
dtype: float64

In [182]:
(sS + other_sS).fillna(0.0)

a     0.0
d     0.0
f    76.0
g     0.0
h     0.0
i     0.0
j     0.0
s     0.0
dtype: float64

In [ ]:
# Most real datasets are read in from an external file format.
aapl = pd.read_csv('AAPL.csv', parse_dates=['Date'], index_col='Date')
aapl.head()

In [ ]:
# Slicing generalizes to two dimensions as you'd expect:
aapl.iloc[:2, :2]

In [ ]:
aapl.loc[pd.Timestamp('2010-02-01'):pd.Timestamp('2010-02-04'), ['Close', 'Volume']]

,Close,Volume
Date,,
2010-02-01,27.818571,187469100.0
2010-02-02,27.980000,174585600.0
2010-02-03,28.461428,153832000.0
2010-02-04,27.435715,189413000.0


# Pandas Review

- Pandas extends numpy with more complex datastructures and algorithms.
- If you understand numpy, you understand 90% of pandas.
- `groupby`, `set_index`, and `unstack` are powerful tools for working with categorical data.
- Avocado prices are surprisingly interesting :)

# Thanks!